# TAIL-RISK FUND: OPTIONS

author: [@sparshsah](https://github.com/sparshsah)

ref: https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/tail-risk-fund.md

In [29]:
import pandas as pd

# https://github.com/sparshsah/foggy-statslib
import sys; sys.path.append("../../../foggy-statslib/")
import foggy_statslib.core as fsc
import foggy_statslib.fin as fsf

## Setting

You hold 2 shares of S, which is currently trading at 100USD.
(S is SPX, but I don't want the fact that SPX is not tradable and also actually at 3000USD to distract you.)

You believe that it is equally likely to:
1. Fall 20pct over the next 6 months (accompanied by a doubling in ivols), Then stay flat to EOY
2. Stay flat over the next 6 months, Then fall 20pct by EOY
3. Stay flat through EOY

Prevailing option-implied vols are around 16pct,
and for simplicity neglect (a) volatility smirks, (b) the risk-free rate, and (c) tcosts.

Crucially: Unlike most single-name equity options which are American-style, the options we consider are European-style.

In [32]:
sigma: Final[float] = 0.16
r: Final[float] = 0

## Question

Should you
* A: Purchase two 2-quarter puts (Rolling into another two if the crash fails to materialize by expiry)?
* B: Purchase two 4-quarter puts?
* C: Purchase one of each?

Assuming you fix your strike at 90USD.
Assume also that you finance your puts with borrowing (not selling shares).

In [43]:
init_value_of_one_put_a = fsf.calc_bsf_option_value(
    tau=2/4,
    K=90,
    S_t=100,
    sigma=sigma,
    r=r,
    put=True,
)
init_value_of_one_put_b = fsf.calc_bsf_option_value(
    tau=4/4,
    K=90,
    S_t=100,
    sigma=sigma,
    r=r,
    put=True,
)
print(f"Premium for each Put: A = ${init_value_of_one_put_a:.2f}, B = ${init_value_of_one_put_b:.2f}.")

Premium for each Put: A = $1.02, B = $2.32.


In [33]:
# dollar value under each scenario of different strategies, by quarter

# SCENARIO 1: S falls -20% over the next 6 months, then stays flat to EOY
v1 = pd.DataFrame(
    {
        # the shares
        "principal": {
            0: 2*100,
            1: 2*100,
            2: 2*80,
            3: 2*80,
            4: 2*80,
        },
        # the hedges
        # a: two 2-quarter puts
        "a": {
            # you own two puts, but you owe what you borrowed to get them
            0: 2*init_value_of_one_put_a - 2*init_value_of_one_put_a,
            1: (
                # your puts are now less valuable, as a quarter has passed with no market movements
                2*fsf.calc_bsf_option_value(
                    tau=1/4,
                    K=90,
                    S_t=100,
                    sigma=sigma,
                    r=r,
                    put=True,
                )
                # you still owe what you borrowed
                - 2*init_value_of_one_put_a
            ),
            # you would exercise both puts, earning $10 a pop
            2: 2*10 - 2*init_value_of_one_put_a,
            3: 2*10 - 2*init_value_of_one_put_a,
            4: 2*10 - 2*init_value_of_one_put_a,
        },
        # b: two 4-quarter puts
        "b": {
            # you own two puts, but you owe what you borrowed to get them
            0: 2*init_value_of_one_put_b - 2*init_value_of_one_put_b,
            1: (
                # your puts are now less valuable, as a quarter has passed with no market movements
                2*fsf.calc_bsf_option_value(
                    tau=3/4,
                    K=90,
                    S_t=100,
                    sigma=sigma,
                    r=r,
                    put=True,
                )
                # you still owe what you borrowed
                - 2*init_value_of_one_put_b
            ),
            # You cannot exercise (we are assuming the puts are European),
            # but the market crash and the vol spike have both made your options more valuable.
            # Since you know we must now be in World 1
            2: 
        },

SyntaxError: incomplete input (2576137096.py, line 20)

In [ ]:
fsf.calc_bsf_option_value(